In [1]:
import xarray as xr
import numpy as np
import scipy as sp
import seastar as ss
import warnings
import os
from _version import __version__

In [2]:
save_folder=os.path.join('D:\data\SEASTAR\SEASTARex\Data\L1c','')
days = ['17','22','25','26']
flights = ['202205' + d for d in days]
file_paths = ss.utils.readers._set_file_paths()
L1b_path = os.path.join(file_paths['l1b_path'],'')

Device name = LIVPCZG-10
Setting local paths...


In [4]:
for flight in flights:
    print('Processing flight ', flight, '...')
    DAR_tracks = ss.utils.readers._read_DAR_config(flight)
    for track in DAR_tracks.keys():
        print('Processing track',track,'...')
        ds_L1b = xr.open_dataset(file_paths['l1b_path'] + flight + '_' + track + '_OSCAR_L1b.nc')
        ds_L1c = ds_L1b.copy(deep=True)
        ds_L1c['Interferogram'].loc[dict({'Antenna': 'Fore'})] = ds_L1b.sel(Antenna='Fore').Interferogram\
                    - -0.0025 * ds_L1b.sel(Antenna='Fore').IncidenceAngleImage - 0.129
        ds_L1c['Interferogram'].loc[dict({'Antenna': 'Aft'})] = ds_L1b.sel(Antenna='Aft').Interferogram\
                    - 0.0043 * ds_L1b.sel(Antenna='Aft').IncidenceAngleImage - -0.2364
        
#         ds_L1c.attrs['Interferogram corrected for Sigma0'] = {'function': 'y = m * x + c',
#                                                   'Fore': 'm = -0.0025, c = 0.129',
#                                          'Aft': 'm = -0.0043, c = -0.2364'}
        rsv = [ss.oscar.level1.compute_radial_surface_velocity(ds_L1c.sel(Antenna=ant))
               for ant in ds_L1c.Antenna.values]
        ds_L1c['RadialSurfaceVelocity'] = xr.concat(rsv, 'Antenna',join='outer')
        print('Saving to disk...')
        ds_L1c.to_netcdf(os.path.join(save_folder, flight + '_' + track +'_OSCAR_L1c.nc'))

Processing flight  20220517 ...
Processing track Track_11 ...
Saving to disk...
Processing track Track_12 ...
Saving to disk...
Processing track Track_13 ...
Saving to disk...
Processing track Track_1 ...
Saving to disk...
Processing track Track_2 ...
Saving to disk...
Processing track Track_1b ...
Saving to disk...
Processing track Track_l1 ...
Saving to disk...
Processing flight  20220522 ...
Processing track Track_1 ...
Saving to disk...
Processing track Track_2 ...
Saving to disk...
Processing track Track_1b ...
Saving to disk...
Processing track Track_11 ...
Saving to disk...
Processing track Track_12 ...
Saving to disk...
Processing track Track_13 ...
Saving to disk...
Processing track Track_14 ...
Saving to disk...
Processing track Track_15 ...
Saving to disk...
Processing track Track_16 ...
Saving to disk...
Processing track Track_17 ...
Saving to disk...
Processing track Track_18 ...
Saving to disk...
Processing track Track_2b ...
Saving to disk...
Processing track Track_l1 ..